# Pipeline推理

`pipeline()` 让使用 `🤗 Hugging Face Hub` 上的任何模型进行任何语言、计算机视觉、语音以及多模态任务的推理都变得非常简单。

即使你对特定的模态没有经验，或者不熟悉模型的源码，你仍然可以使用pipeline()进行推理！

本教程说明：

1. 如何使用 pipeline() 进行推理。
2. 如何使用特定的 tokenizer(分词器) 或 model(模型)。
3. 如何使用 pipeline() 进行音频、视觉和多模态任务的推理。

```
请查看[pipeline()](https://huggingface.co/docs/transformers/v4.44.2/zh/main_classes/pipelines#transformers.pipeline)文档以获取已支持的任务和可用参数的完整列表。
```

## Pipeline使用

虽然每种任务都有一个关联的 pipeline，但我们可以使用通用的 `pipeline()` 方法，其中包含所有特定任务的 pipelines，**能够根据任务类型自动加载一个默认模型和一个能够进行任务推理的预处理类**。

让我们以使用 pipeline() 进行自动语音识别（ASR）或语音转文本为例。

1. 首先，创建一个 pipeline() 并指定推理任务：

In [ ]:
from transformers import pipeline

transcriber = pipeline(task="automatic-speech-recognition", device=0) # 任务：自动语音识别（ASR）

2. 将你的输入传递给 pipeline()。对于语音识别，通常是输入一个音频文件：

In [ ]:
transcriber("../resources/speech/mlk.flac")

3. 如果你在 MAC OS 中运行，可能会遇到错误信息 "ffmpeg was not found but is required to load audio files from filename"。这表示你的环境中缺少 `ffmpeg`，这是一个用于处理音频和视频文件的强大工具。在 macOS 上，你可以使用 Homebrew 来安装 ffmpeg。

In [ ]:
brew install ffmpeg

如果结果不符合你的预期，还可以在 Hub 上查看一些[最受欢迎的自动语音识别模型](https://huggingface.co/models?pipeline_tag=automatic-speech-recognition&sort=trending)，看看是否可以获得更好的转录结果。

让我们尝试来自 OpenAI 的 [Whisper large-v2](https://huggingface.co/openai/whisper-large) 模型。Whisperb 比 Wav2Vec2 晚2年发布，使用接近10倍的数据进行了训练。因此，它在大多数下游基准测试上击败了 Wav2Vec2。 它还具有预测标点和大小写的附加优势，而 Wav2Vec2 则无法实现这些功能。

让我们在这里尝试一下，看看它的表现如何：

In [ ]:
transcriber = pipeline(task="automatic-speech-recognition", model="openai/whisper-large-v2")

transcriber("../resources/speech/mlk.flac")

现在这个结果看起来更准确了！

要进行深入的 Wav2Vec2 与 Whisper 比较，请参阅[音频变换器课程](https://huggingface.co/learn/audio-course/chapter5/asr_models)。 

建议在 Hub 上查看不同语言的模型，以及专业领域的模型等。可以在 Hub 上直接查看并比较模型的结果，以确定是否适合或处理边缘情况是否比其他模型更好。

如果没有找到适用于你的用例的模型，可以通过[训练](https://huggingface.co/docs/transformers/v4.44.2/zh/training)来获得适合自己的模型！

如果你有多个输入，可以将输入作为列表传递：

In [ ]:
transcriber(
    [
        "../resources/speech/mlk.flac",
        "../resources/speech/1.flac",
    ]
)

从一个模型切换到另一个模型非常琐碎的过程，但使用 Pipelines 能够非常迅速的切换模型，所以它非常适合用于测试。

但是，通过一些方法还可以将 Pipelines 用于大型工作负载而不仅仅是测试。详细请查看以下指南，深入探讨如何迭代整个数据集或在 Web 服务器中使用 Pipelines：

1. [在数据集上使用流水线](https://huggingface.co/docs/transformers/v4.44.2/zh/pipeline_tutorial#using-pipelines-on-a-dataset)
2. [在Web服务器中使用流水线](https://huggingface.co/docs/transformers/v4.44.2/zh/pipeline_webserver)

## 参数

`pipeline()` 支持了许多参数；有些是适用于特定任务的，有些则是适用于所有的 pipeline。通常情况下，你可以在任何地方指定对应的参数：

In [ ]:
transcriber = pipeline(model="openai/whisper-large-v2", my_parameter=1)

out = transcriber(...)  # 这将使用 `my_parameter=1`。
out = transcriber(..., my_parameter=2)  # 这将覆盖并使用 `my_parameter=2`。
out = transcriber(...)  # 这将返回到使用 `my_parameter=1`。

**让我们查看其中的三个重要参数：**

### 设备

如果你设置 `device=n`，pipeline 会自动将模型加载到指定的设备上（包括存储模型的权重）。支持 PyTorch 或 Tensorflow。

In [ ]:
transcriber = pipeline(model="openai/whisper-large-v2", device=0)

如果你使用的是 PyTorch ，且觉得模型对于单个GPU来说过于庞大，则可以设置 `device_map="auto"` 使其自动确定如何加载和存储模型的权重。使用 `device_map` 参数需要安装 `🤗 Accelerate` 软件包：

In [ ]:
pip install --upgrade accelerate

以下代码会自动在各个设备上加载和存储模型权重：

In [ ]:
transcriber = pipeline(model="openai/whisper-large-v2", device_map="auto")

**请注意，如果设置了 `device_map="auto"`，在实例化 pipeline 时则不需要设置 `device=n` 参数，否则可能会遇到一些意外的状况！**

### 批量大小

默认情况下，pipelines 不会进行批量推理，因为批处理不一定更快，在某些情况下可能会更慢，原因可查阅[详细解释](https://huggingface.co/docs/transformers/main_classes/pipelines#pipeline-batching)。

想在用例中使用批量推理，如下：

In [ ]:
import os

audio_filenames = [os.path.join('../resources/speech', f"{i}.flac") for i in range(1, 5)]

transcriber = pipeline(model="openai/whisper-large-v2", device=0, batch_size=2)
texts = transcriber(audio_filenames)

以上代码会在提供的4个音频文件上运行 pipeline，设置 `batch_size=2` 表示 pipeline 会将它们以每2个为一组的批次传递给模型（若模型加载在GPU上，此时批处理可能更有作用），不需要编写额外的代码。

**输出的结果会始终与没有批处理时收到的结果相一致，因为批量推理只是一种帮助您更快地运行 pipeline 的方式，你可以将它理解为是 pipeline 的带宽大小。**

同时，使用 pipeline() 能够帮助我们减轻工作量，降低批处理的复杂性。因为对于某些 pipeline ，需要将单个项目（如长音频文件）分成多个部分后才能供模型处理。**pipeline() 会在需要时自动执行 chunk batching（分块处理）。**

### 任务特定参数

transformers 为所有任务的 pipeline 都提供了特定的参数，这些参数使得 pipeline 更具灵活性，能够帮助你更好地完成工作。 

例如，`transformers.AutomaticSpeechRecognitionPipeline.call()` 方法就具有一个 `return_timestamps` 参数，能够在模型推断出文本的同时输出各个句子发音的时间戳，对于需要添加字幕的视频很有帮助：

In [ ]:
transcriber = pipeline(model="openai/whisper-large-v2", return_timestamps=True)

transcriber("https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/mlk.flac")

每个任务都有许多可用的参数，因此请查看每个任务的API参考。

例如，在为整部电影或长达一小时的视频添加字幕时，需要模型处理非常长的音频文件，但这通常是模型无法单独处理的，可以通过`AutomaticSpeechRecognitionPipeline` 设置 `chunk_length_s` 参数来控制音频处理的分块长度，这非常有帮助：

In [ ]:
transcriber = pipeline(model="openai/whisper-large-v2", chunk_length_s=30, return_timestamps=True)

transcriber("https://huggingface.co/datasets/sanchit-gandhi/librispeech_long/resolve/main/audio.wav")

如果你找不到任何一个有帮助的参数，欢迎[提出请求](https://github.com/huggingface/transformers/issues/new?assignees=&labels=feature&template=feature-request.yml)！